In [34]:
import json

In [35]:
few_shots = []

# 266 voldemort 10099.json

In [36]:
message = "Fix transactional bug in mysql--transaction must be committed."
diff = """
@@ -1,58 +1,59 @@
     public void put(ByteArray key, Versioned<byte[]> value) throws PersistenceFailureException {
         StoreUtils.assertValidKey(key);
         boolean doCommit = false;
         Connection conn = null;
         PreparedStatement insert = null;
         PreparedStatement select = null;
         ResultSet results = null;
         String insertSql = "insert into " + name + " (key_, version_, value_) values (?, ?, ?)";
         String selectSql = "select key_, version_ from " + name + " where key_ = ?";
         try {
             conn = datasource.getConnection();
             conn.setAutoCommit(false);
 
             // check for superior versions
             select = conn.prepareStatement(selectSql);
             select.setBytes(1, key.get());
             results = select.executeQuery();
             while(results.next()) {
                 byte[] thisKey = results.getBytes("key_");
                 VectorClock version = new VectorClock(results.getBytes("version_"));
                 Occured occured = value.getVersion().compare(version);
                 if(occured == Occured.BEFORE)
                     throw new ObsoleteVersionException("Attempt to put version "
                                                        + value.getVersion()
                                                        + " which is superceeded by " + version
                                                        + ".");
                 else if(occured == Occured.AFTER)
                     delete(conn, thisKey, version.toBytes());
             }
 
             // Okay, cool, now put the value
             insert = conn.prepareStatement(insertSql);
             insert.setBytes(1, key.get());
             VectorClock clock = (VectorClock) value.getVersion();
             insert.setBytes(2, clock.toBytes());
             insert.setBytes(3, value.getValue());
             insert.executeUpdate();
+            doCommit = true;
         } catch(SQLException e) {
             if(e.getErrorCode() == MYSQL_ERR_DUP_KEY || e.getErrorCode() == MYSQL_ERR_DUP_ENTRY) {
                 throw new ObsoleteVersionException("Key or value already used.");
             } else {
                 throw new PersistenceFailureException("Fix me!", e);
             }
         } finally {
             if(conn != null) {
                 try {
                     if(doCommit)
                         conn.commit();
                     else
                         conn.rollback();
                 } catch(SQLException e) {}
             }
             tryClose(results);
             tryClose(insert);
             tryClose(select);
             tryClose(conn);
         }
     }
"""
explanation = ""
answer = "Buggy"

In [37]:
few_shots.append(
    {"Message": message, "Diff": diff, "Explanation": explanation, "Answer": answer},
)

# 265 voldemort 10087.json

In [38]:
message = "Fix transactional bug in mysql--transaction must be committed."
diff = """
@@ -1,5 +1,5 @@
     public void create() {
-        execute("create table "
-                + getName()
-                + " (key_ varbinary(200) not null, version_ varbinary(200) not null, value_ blob, primary key(key_, version_))");
+        execute("create table " + getName()
+                + " (key_ varbinary(200) not null, version_ varbinary(200) not null, "
+                + " value_ blob, primary key(key_, version_)) engine = InnoDB");
     }
"""
explanation = ""
answer = "NotBuggy"

In [39]:
few_shots.append(
    {"Message": message, "Diff": diff, "Explanation": explanation, "Answer": answer},
)

# 75 PMD 2899.json

In [40]:
message = """fixed bug in CPD directory scanning


git-svn-id: https://pmd.svn.sourceforge.net/svnroot/pmd/trunk@660 51baf565-9d33-0410-a72c-fc3788e3496d"""
diff = """
@@ -1,12 +1,14 @@
     private void scanDirectory(File dir, List list, boolean recurse) {
      FilenameFilter filter = new JavaFileOrDirectoryFilter();
      String[] possibles = dir.list(filter);
      for (int i=0; i<possibles.length; i++) {
         File tmp = new File(dir + System.getProperty("file.separator") + possibles[i]);
-        if (recurse && tmp.isDirectory()) {
-           scanDirectory(tmp, list, true);
+        if (tmp.isDirectory()) {
+            if (recurse) {
+                scanDirectory(tmp, list, true);
+            }
         } else {
            list.add(new File(dir + System.getProperty("file.separator") + possibles[i]));
         }
      }
     }
"""
explanation = ""
answer = "Buggy"

In [41]:
few_shots.append(
    {"Message": message, "Diff": diff, "Explanation": explanation, "Answer": answer},
)

# 76 PMD 2494.json

In [42]:
message = """fixed bug in CPD directory scanning


git-svn-id: https://pmd.svn.sourceforge.net/svnroot/pmd/trunk@660 51baf565-9d33-0410-a72c-fc3788e3496d"""
diff = """
@@ -1,23 +1,24 @@
     private void go() {
         try {
             CPD cpd = new CPD();
             cpd.setListener(this);
             cpd.setMinimumTileSize(Integer.parseInt(minimumLengthField.getText()));
             addingTokensBar.setStringPainted(true);
             if (rootDirectoryField.getText().endsWith(".java")) {
                 cpd.add(new File(rootDirectoryField.getText()));
             } else {
                 if (recurseCheckbox.isSelected()) {
                     cpd.addRecursively(rootDirectoryField.getText());
                 } else {
                     cpd.addAllInDirectory(rootDirectoryField.getText());
                 }
             }
             cpd.go();
             CPDRenderer renderer = new TextRenderer();
             resultsTextArea.setText("");
             resultsTextArea.append(renderer.render(cpd));
         } catch (IOException ioe) {
             ioe.printStackTrace();
+            JOptionPane.showMessageDialog(null, "Halted due to " + ioe.getClass().getName() + "; " + ioe.getMessage());
         }
     }
"""
explanation = ""
answer = "NotBuggy"

In [43]:
few_shots.append(
    {"Message": message, "Diff": diff, "Explanation": explanation, "Answer": answer},
)

# 280 weka 4596.json

In [44]:
message = """Fixed a bug so it can be used without setting any options.


git-svn-id: https://svn.cms.waikato.ac.nz/svn/weka/trunk@1855 e0a1b77d-ad91-4216-81b1-defd5f83fa92
"""
diff = """
@@ -1,7 +1,8 @@
     public void setOptions(String[] options) throws Exception {
         
         setNormalizeWordWeights(Utils.getFlag('N', options));
         
         String val = Utils.getOption('S', options);
-        setSmoothingParameter(Double.parseDouble(val));
+        if(val.length()!=0)
+          setSmoothingParameter(Double.parseDouble(val));
     }
"""
explanation = ""
answer = "Buggy"

In [45]:
few_shots.append(
    {"Message": message, "Diff": diff, "Explanation": explanation, "Answer": answer},
)

# 15 sprint-boot 9497.json

In [46]:
message = """Add support for custom ProtocolResolver with Devtools

Prior to this commit, custom `ProtocolResolvers` set on the
`ApplicationContext` were lost when Devtools is used as the customized
`ResourceLoader` did not copy any customization made to the default
resource loader.

This commit makes sure to copy any `ProtocolResolver` set on the context.

Closes gh-9331
"""
diff = """
@@ -1,10 +1,9 @@
 		public ResourcePatternResolver getResourcePatternResolver(
 				ApplicationContext applicationContext, ResourceLoader resourceLoader) {
 			if (applicationContext instanceof WebApplicationContext) {
 				return new ServletContextResourcePatternResolver(resourceLoader == null
-						? new WebApplicationContextResourceLoader(
-								(WebApplicationContext) applicationContext)
+						? createResourceLoader((WebApplicationContext) applicationContext)
 						: resourceLoader);
 			}
 			return super.getResourcePatternResolver(applicationContext, resourceLoader);
 		}
"""
explanation = ""
answer = "NotBuggy"

In [47]:
few_shots.append(
    {"Message": message, "Diff": diff, "Explanation": explanation, "Answer": answer},
)

# Create json

In [48]:
json.dump(few_shots,open("./data/FewShots.json", "w"))